In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder

In [3]:
raw_train_data = pd.read_csv('train_5CLrC8b/train.csv')
raw_test_data = pd.read_csv('test_Jwt0MQH/test.csv')

In [4]:
def dayDiff(col1,col2):
    col=[]
    for i in range(len(col1)):
        st=datetime.strptime(col1[i], '%d/%m/%y')
        et=datetime.strptime(col2[i], '%d/%m/%y')
        col.append((et-st).days)
    return col

In [5]:
raw_train_data['booking_days']=dayDiff(raw_train_data['booking_date'],raw_train_data['checkin_date'])
raw_train_data['staying_days']=dayDiff(raw_train_data['checkin_date'],raw_train_data['checkout_date'])
raw_test_data['booking_days']=dayDiff(raw_test_data['booking_date'],raw_test_data['checkin_date'])
raw_test_data['staying_days']=dayDiff(raw_test_data['checkin_date'],raw_test_data['checkout_date'])



In [6]:
result=raw_train_data['amount_spent_per_room_night_scaled']
x_test_resId=raw_test_data['reservation_id']
raw_train_data.drop(['reservation_id','memberid','booking_date','checkin_date','checkout_date','amount_spent_per_room_night_scaled'],1, inplace=True)
raw_test_data.drop(['reservation_id','memberid','booking_date','checkin_date','checkout_date'],1, inplace=True)
raw_train_data['amount_spent_per_room_night_scaled']=result

In [7]:
def handle_non_numerical_data(df):
    columns = df.columns.values
    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]

        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1

            df[column] = list(map(convert_to_int, df[column]))

    return df

In [8]:
raw_train_data=handle_non_numerical_data(raw_train_data)
raw_test_data=handle_non_numerical_data(raw_test_data)

In [9]:
train_data = raw_train_data.fillna(0).values
test_data = raw_test_data.fillna(0).values

In [20]:
x_train,y_train,x_test=train_data[:,:20],train_data[:,20:],test_data

In [31]:
from sklearn.linear_model import Ridge
reg = Ridge().fit(x_train, y_train)

In [32]:
y_test = reg.predict(x_test)

In [33]:
y_test


array([[7.82304857],
       [7.42913171],
       [7.53563284],
       ...,
       [7.98463153],
       [7.2262242 ],
       [7.51172425]])

In [35]:
np.savetxt("submission.csv", y_test, delimiter=",")

In [29]:
test_data.shape

(146765, 20)